# Ludność wojewdztw

In [ ]:
# %matplotlib inline
import os
import re
import zipfile
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import geopandas  # depends on descartes
import numpy as np
import pandas as pd
import seaborn as sns


def read_shape_from_zip(zip_filename):
    z = zipfile.ZipFile(zip_filename, mode="r")
    dbf_name = list(filter(
        lambda x: re.match(r'.*\.dbf$', os.path.basename(x)),
        z.namelist()
    ))[0]
    z.close()
    return geopandas.read_file(dbf_name, encoding='utf-8', vfs='zip://'+zip_filename)


fn_woj = '../gis_data/wojewodztwa.zip'
woj = read_shape_from_zip(fn_woj)
woj['JPT_KOD_JE'] = woj['JPT_KOD_JE'].astype(np.int64)
woj.plot()

fn_ludnosc = '../data/ludnosc_wojewodztw.csv'

df = pd.read_csv(
    fn_ludnosc
)

sns.set(style="white", font_scale=1.2)
df = pd.merge(woj, df, on='JPT_KOD_JE')
ax = df.plot(column='Ludność Ogółem', cmap='OrRd')
ax.set_title("Ludność w województwach")
ax.grid(False)
ax.set_axis_off()
plt.tight_layout()
plt.show()

r = requests.get('http://bdl.stat.gov.pl/api/v1/variables?subject-id=P2914')
r = requests.get('https://bdl.stat.gov.pl/api/v1/data/by-variable/199203?format=json&year=2020')
r = requests.get('https://bdl.stat.gov.pl/api/v1/data/by-unit/011200000000?format=json&var-id=199203&year=2020')
r = requests.get('https://bdl.stat.gov.pl/api/v1/data/localities/by-variable/199203?format=json&unit-parent-id=000000000000&year=2020')

# liczba mieszkanow dla alopolskiego za 2020
requests.get('https://bdl.stat.gov.pl/api/v1/data/by-unit/011200000000?format=json&var-id=199203&year=2020')

# dane dla wojewodztw:
requests.get('https://bdl.stat.gov.pl/api/v1/data/by-variable/199203?year=2020&unit-level=2&aggregate-id=1&page=0&page-size=10')
requests.get('https://bdl.stat.gov.pl/api/v1/data/by-variable/199203?year=2020&unit-level=2&aggregate-id=1').json()
requests.get('https://bdl.stat.gov.pl/api/v1/data/by-variable/199203?year=2020&unit-level=2').json()